In [27]:
!pip install neo4j
!pip install neo4j-driver
!pip install neobolt

In [28]:
from neo4j import GraphDatabase, basic_auth

In [29]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "hsyHSY123"))
session = driver.session()

In [30]:
print("Question1: Find all producers that produced the movie When Harry Met Sally. ")
Harry_Met_Sally_producer = '''
MATCH (producer:Person)-[:PRODUCED]->(harry:Movie {title:'When Harry Met Sally'}) 
RETURN producer.name
'''

results = session.run(Harry_Met_Sally_producer)
for record in results:
    print(record["producer.name"])

Question1: Find all producers that produced the movie When Harry Met Sally. 
Rob Reiner
Nora Ephron
Rob Reiner
Nora Ephron


In [31]:
print("Question2: Find directors who have directed more than 2 movies.  ")
director_more_than_2_movies = '''
MATCH (director:Person)-[:DIRECTED]->(movie:Movie)
WITH director, COUNT(movie) AS moviesDirected 
WHERE moviesDirected > 2
RETURN director.name AS name, moviesDirected
ORDER BY moviesDirected DESC
'''

results = session.run(director_more_than_2_movies)
for record in results:
    print(record["name"],record["moviesDirected"])

Question2: Find directors who have directed more than 2 movies.  
Larry Wachowski 5
Lilly Wachowski 5
Lana Wachowski 5
Lilly Wachowski 5
Rob Reiner 3
Ron Howard 3
Rob Reiner 3
Ron Howard 3


In [32]:
print("Question3: Find the actors who acted in 5+ movies and the movies in which they acted. ")
actor_more_than_5_movies = '''
MATCH (actor:Person)-[:ACTED_IN]->(movie:Movie)
WITH actor, COUNT(movie) AS moviesActed 
WHERE moviesActed > 5
RETURN actor.name AS name, moviesActed
ORDER BY moviesActed DESC
'''

results = session.run(actor_more_than_5_movies)
for record in results:
    print(record["name"],record["moviesActed"])

Question3: Find the actors who acted in 5+ movies and the movies in which they acted. 
Tom Hanks 12
Tom Hanks 12
Keanu Reeves 7
Keanu Reeves 7


In [33]:
print("Question4: Find the people who were the producer, writer, AND director (the same person who held all 3 roles) at the same time on a film.  ")
at_the_same_movie = '''
MATCH (p:Person)-[:PRODUCED]->(m:Movie),
      (p)-[:WROTE]->(m),
      (p)-[:DIRECTED]->(m)
RETURN p.name AS name, m.title AS title
'''

results = session.run(at_the_same_movie)
for record in results:
    print(record["name"],"  Movie Title:",record["title"])

Question4: Find the people who were the producer, writer, AND director (the same person who held all 3 roles) at the same time on a film.  
Cameron Crowe   Movie Title: Jerry Maguire
Nancy Meyers   Movie Title: Something's Gotta Give
Cameron Crowe   Movie Title: Jerry Maguire
Nancy Meyers   Movie Title: Something's Gotta Give


In [34]:
print("Question5: Find all the actors that are exactly 3 degrees away from Tom Hanks.")
degree_from_Tom = '''
MATCH (tom:Person {name: "Tom Hanks"})-[:ACTED_IN]->(m1:Movie)<-[:ACTED_IN]-(p1:Person)-[:ACTED_IN]->(m2:Movie)<-[:ACTED_IN]-(p2:Person)-[:ACTED_IN]->(m3:Movie)<-[:ACTED_IN]-(p3:Person)
WHERE tom <> p1 AND p1 <> p2 AND p2 <> p3 AND tom <> p3 AND tom <> p2
AND m1 <> m2 AND m2 <> m3 AND m1 <> m3
RETURN DISTINCT p3.name AS name
LIMIT 20
'''

results = session.run(degree_from_Tom)
for record in results:
    print(record["name"])

Question5: Find all the actors that are exactly 3 degrees away from Tom Hanks.
Michael Sheen
Frank Langella
Oliver Platt
Kevin Bacon
Jerry O'Connell
Jay Mohr
Cuba Gooding Jr.
Jonathan Lipnicki
Renee Zellweger
Kelly Preston
Regina King
Tom Cruise
Ethan Hawke
Rick Yune
Max von Sydow
Bonnie Hunt
Patricia Clarkson
James Cromwell
Michael Clarke Duncan
David Morse


In [35]:
print("Question6: Update database to reflect new name Larry Wachowski for director Lana Wachowski.")
update = '''
MATCH (actor:Person {name: "Lana Wachowski"})
SET actor.name = "Larry Wachowski"
RETURN actor.name as name
'''

results = session.run(update)
for record in results:
    print(record["name"])

Question6: Update database to reflect new name Larry Wachowski for director Lana Wachowski.
Larry Wachowski


In [36]:
print("Question7: Create a section of fun movies whose summary contains the keyword fun.")
fun_movie = '''
MATCH (p:Person)-[review:REVIEWED]->(m:Movie)
WHERE review.summary CONTAINS "fun"
RETURN DISTINCT m.title AS name
'''

results = session.run(fun_movie)
for record in results:
    print(record["name"])

Question7: Create a section of fun movies whose summary contains the keyword fun.
The Replacements


In [37]:
print("Question8:List all the release years and count all movies released in that year.")
movie_by_year = '''
MATCH (m:Movie)
WITH m.released AS year, COUNT(m) AS num
RETURN year, num
ORDER BY year ASC
'''

results = session.run(movie_by_year)
for record in results:
    print(record["year"]," ",record["num"])

Question8:List all the release years and count all movies released in that year.
1975   2
1986   4
1990   2
1992   8
1993   2
1995   4
1996   6
1997   4
1998   6
1999   8
2000   6
2003   6
2004   2
2006   6
2007   2
2008   4
2009   2
2012   2


In [38]:
print("Question9: List all the movies whose rating >= 90.")
movie_rating_90 = '''
MATCH (p:Person)-[review:REVIEWED]->(m:Movie)
WITH m.title as title, AVG(review.rating) AS avg_rating
WHERE avg_rating >= 90
RETURN title,avg_rating
ORDER BY avg_rating DESC
'''

results = session.run(movie_rating_90)
for record in results:
    print(record["title"],record["avg_rating"])

Question9: List all the movies whose rating >= 90.
Cloud Atlas 95.0
Jerry Maguire 92.0


In [39]:
print("Question10: List the top producers who are most likely to entertain new movies production hint: Producers with most movies produced.")
producer_produced_most = '''
MATCH (p:Person)-[:PRODUCED]->(m:Movie)
WITH p.name as name, COUNT(m) AS num
RETURN name, num
ORDER BY num DESC
LIMIT 3
'''

results = session.run(producer_produced_most)
for record in results:
    print(record["name"],"(with the number of movies produced:",record["num"],")")

Question10: List the top producers who are most likely to entertain new movies production hint: Producers with most movies produced.
Joel Silver (with the number of movies produced: 12 )
Lilly Wachowski (with the number of movies produced: 4 )
Larry Wachowski (with the number of movies produced: 4 )


In [40]:
session.close()